# 加载谷歌硬盘


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 安装运行环境

以下内容是安装运行环境

In [ ]:
!sudo apt-get update \
 && sudo apt-get install -y \
  libxcb-icccm4 -y \
  libxcb-image0 -y \
  libxcb-keysyms1 -y \
  libxcb-randr0 -y \
  libxcb-render-util0 -y \
  libxcb-shape0 -y \
  libxcb-xfixes0 -y \
  libxcb-xinerama0 -y \
  libxcb-xkb1 -y \
  libxkbcommon-x11-0 -y\
  ffmpeg -y\
  libsm6 -y\
  libxext6 -y\
  freeglut3-dev\
 && sudo rm -rf /var/lib/apt/lists/* 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,069 kB]
Hit:14 http://ppa.launchpad.net/graphi

In [ ]:
!pip3 install numpy \
pyqt5 \
pillow \
gym-retro \
gym==0.21.0 \
matplotlib \
stable-baselines3[extra] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 KB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 34.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
    

导入游戏ROM

In [ ]:
! python3  -m retro.import ./drive/MyDrive/Contra_train/Contra-Nes/

Importing Contra-Nes
Imported 1 games


导入运行库

In [ ]:
import random
import retro
import gym

from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
# 多线程
from stable_baselines3.common.vec_env import VecMonitor
# 随机种子
from stable_baselines3.common.utils import set_random_seed

from stable_baselines3.common.env_util import make_vec_env

import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback

import cv2
import pandas as pd

from typing import Callable

import time

import torch


切换到硬盘目录

In [ ]:
%cd /content/drive/MyDrive/Contra_train/


/content/drive/MyDrive/Contra_train


In [ ]:

SCRIPT_DIR = os.path.dirname(os.path.abspath("./Contra-Nes"))

retro.data.Integrations.add_custom_path(
    os.path.join(SCRIPT_DIR, "Contra-Nes/")
)
rom_path = os.path.join(SCRIPT_DIR, "Contra-Nes/")


CHECK_FREQ_NUMB = 10000
TOTAL_TIMESTEP_NUMB = 2000000
# LEARNING_RATE = 0.00005
LEARNING_RATE = 0.000003
N_STEPS = 2048
GAMMA = 0.99
BATCH_SIZE = 64
N_EPOCHS = 10
DOWN_SAMPLE_RATE = 3
SKIP_NUMB = 4
EPISODE_NUMBERS = 20

CHECKPOINT_DIR = './training/'
LOG_DIR = './logs/'
REWARD_LOG_FILENAME = 'reward_log.csv'

with open(REWARD_LOG_FILENAME, 'a') as f:
    print('timesteps,reward,bestreward', file=f)


# 计算奖励函数
class DeadlockEnv(gym.Wrapper):
    def __init__(self, env, threshold=10):
        super().__init__(env)
        self.last_lives = 2
        self.count = 0
        self.threshold = threshold
        self.last_xscroll = 0
        self.score = 0
        # 初始武器
        self.Weapon = 0

    def reset(self, **kwargs):
        self.score = 0
        self.last_lives = 2
        self.count = 0
        self.last_xscroll = 0
        # 定义一个空的屏幕
        self.previous_frame = np.zeros((74, 80, 1))
        # 人物生成位置随机
        numbers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
        random_number = random.choice(numbers)
        self.env.load_state("Level4-"+random_number)
        self.enemy_hp_list = [0] * 11

        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        reward = 0
        # self.env.render()
        # print(state.shape)
        ifdie = info['die']
        lives = info['lives']
        score = info['score']
        xpos = info['xpos']
        xscroll = info['xscroll']
        boss_defeated = info['beat_boss']
        Weapon = info['Weapon']

        if lives == 0:
            reward = 0
            done = True

        # 打中敌人加分
        for i in range(1, 10):
            var_name = 'ehp' + str(i)
            # print(var_name+":"+str(info[var_name]))
            if info[var_name]>1 and info[var_name]<self.enemy_hp_list[i]: 
                # print("打中敌人加分+1")
                reward += 10
                self.enemy_hp_list[i]=info[var_name]
            elif info[var_name]>self.enemy_hp_list[i]:
                self.enemy_hp_list[i]=info[var_name]

        #分数提成 
        if score > self.score:
            reward += (score-self.score)/100
            self.score = score
        # 获取S弹加分
        # Weapon 3 或者19 是s弹
        if Weapon == 3 and Weapon != self.Weapon:
            reward += 10
            self.Weapon = Weapon
        elif Weapon == 19 and Weapon != self.Weapon:
            reward += 10
            self.Weapon = Weapon
        elif Weapon == 16 and Weapon != self.Weapon:
            reward += 10
            self.Weapon = Weapon

        # Animation==85 被电了
        if info['Animation'] == 85:
            reward -= 0.01
            # reward = 0
            # done = True

        # 防止卡死
        if xscroll == self.last_xscroll:
            if xscroll != 1208:
                reward -= 0.01
                self.count += 1
            elif xscroll == 1208:
                self.count = 0
        else:
            self.count = 0
				
        if self.count >= 3200 :
            reward = 0
            print("超时了")
            done = True

            # # 防止卡死
        # if xscroll ==1208:
        #     self.count += 1
        # if self.count >= 8000:
        #     # reward -= 10
        #     done = True

        # if xscroll == self.last_xscroll:
        #     reward -= 0.01

        if xscroll != self.last_xscroll:
            # xscroll in [1, 2, 3, 4, 5]:
            # if xscroll in [1, 2, 3, 4, 5]:
            # 	reward +=15
            # else:
            reward += 3
            self.last_xscroll = xscroll

        if ifdie != 1:
            reward -= 0.01
        # 通关得分
        if boss_defeated == 8:
            reward += 5000
        # print(reward)
        # time.sleep(0.01)
        return state, reward, done, info

# 跳过画面每2真保留一个画面，节省计算时间


class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if done:
                break
        return obs, reward, done, info


class Downsample(gym.ObservationWrapper):
    def __init__(self, env, ratio):
        """
        Downsample images by a factor of ratio
        """
        gym.ObservationWrapper.__init__(self, env)
        (oldh, oldw, oldc) = env.observation_space.shape
        newshape = (oldh//ratio, oldw//ratio, oldc)
        self.observation_space = gym.spaces.Box(low=0, high=255,
                                                shape=newshape, dtype=np.uint8)

    def observation(self, frame):
        height, width, _ = self.observation_space.shape
        frame = cv2.resize(frame, (width, height),
                           interpolation=cv2.INTER_AREA)
        if frame.ndim == 2:
            frame = frame[:, :, None]
        # 计算和上一帧的差值
        frame_delta = frame - self.previous_frame
        self.previous_frame = frame
        # 将画面还原为普通的。方便人类观察
        # result = np.where(frame_delta != 0, frame, 0)
        # # 绘制画面
        # cv2.imshow('game',np.array(result))
        # key = cv2.waitKey(10)
        return frame_delta


class Discretizer(gym.ActionWrapper):
    """
    Wrap a gym environment and make it use discrete actions.
    Args:
        combos: ordered list of lists of valid button combinations
    """

    def __init__(self, env, combos):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.MultiBinary)
        buttons = env.unwrapped.buttons
        self._decode_discrete_action = []
        for combo in combos:
            arr = np.array([False] * env.action_space.n)
            for button in combo:
                arr[buttons.index(button)] = True
            self._decode_discrete_action.append(arr)

        self.action_space = gym.spaces.Discrete(
            len(self._decode_discrete_action))

    def action(self, act):
        return self._decode_discrete_action[act].copy()


class SonicDiscretizer(Discretizer):
    """
    Use Sonic-specific discrete actions
    based on https://github.com/openai/retro-baselines/blob/master/agents/sonic_util.py
    """
    # B是子弹

    def __init__(self, env):
        super().__init__(env=env, combos=[['UP', 'B'], ['DOWN', 'B'], [
            'LEFT', 'B'], ['RIGHT', 'B'], ['A'], ['B']])


# env = retro.make(game='Contra-Nes' ,state="Level1",record="./record/.")


env = retro.make(game=rom_path, state="Level4-0",
                 inttype=retro.data.Integrations.CUSTOM_ONLY)

# # 人物生成位置随机
# numbers = [0,1,2,3,4,5]
# random_number = random.choice(numbers)
# rom = env.data.set_value
# rom("xscroll", random_number)

# 限制按键
env = SonicDiscretizer(env)
# 计算奖励函数
env = DeadlockEnv(env)
# 跳过一阵的画面
env = SkipFrame(env, skip=SKIP_NUMB)
monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir, exist_ok=True)
env = Monitor(env, monitor_dir)

env = GrayScaleObservation(env, keep_dim=True)
# 把画面的画质除以3
# (224, 240, 1)
# 变成
# (74, 80, 1)
env = Downsample(env, DOWN_SAMPLE_RATE)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

global best_score
best_score = 0


class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(
                self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

            total_reward = [0] * EPISODE_NUMBERS
            total_time = [0] * EPISODE_NUMBERS
            best_reward = 0
            global best_score
            for i in range(EPISODE_NUMBERS):
                state = env.reset()  # reset for each new trial
                done = False
                total_reward[i] = 0
                total_time[i] = 0
                while not done and total_time[i] < 10000:
                    action, _ = model.predict(state)
                    last_state = state
                    state, reward, done, info = env.step(action)
                    total_reward[i] += reward[0]
                    total_time[i] += 1

                if total_reward[i] > best_reward:
                    best_reward = total_reward[i]
                    # 绘制画面
                    if best_reward > best_score:
                        best_score = best_reward
                        cv2.imwrite('last3.png', np.array(
                            last_state[0][:, :, 3]))
                        cv2.imwrite('last2.png', np.array(
                            last_state[0][:, :, 2]))
                        cv2.imwrite('last1.png', np.array(
                            last_state[0][:, :, 1]))
                        cv2.imwrite('last0.png', np.array(
                            last_state[0][:, :, 0]))
                    best_epoch = self.n_calls

                state = env.reset()  # reset for each new trial

            print('time steps:', self.n_calls, '/', TOTAL_TIMESTEP_NUMB)
            print('average reward:', (sum(total_reward) / EPISODE_NUMBERS),
                  'average time:', (sum(total_time) / EPISODE_NUMBERS),
                  'best_reward:', best_reward,
                  )
            with open(REWARD_LOG_FILENAME, 'a') as f:
                print(self.n_calls, ',', sum(total_reward) /
                      EPISODE_NUMBERS, ',', best_reward, file=f)
        return True


class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, save_model_dir, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = os.path.join(save_model_dir, 'best_model/')
        self.best_mean_reward = -np.inf

    # def _init_callback(self) -> None:
    def _init_callback(self):
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    # def _on_step(self) -> bool:
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            print('self.n_calls: ', self.n_calls)
            model_path1 = os.path.join(
                self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path1)
        return True


callback = TrainAndLoggingCallback(
    check_freq=CHECK_FREQ_NUMB, save_path=CHECKPOINT_DIR)
# This is the AI model started
# model = PPO('CnnPolicy', env, verbose=0, tensorboard_log=LOG_DIR, learning_rate=LEARNING_RATE, n_steps=N_STEPS,
#             batch_size=BATCH_SIZE, n_epochs=N_EPOCHS, gamma=GAMMA)
# 'learning_rate': 9.434717363652453e-05,

model_params = {
    'n_steps': 5952,
    'gamma': 0.829362230935383,
    # 'learning_rate': 8e-06,
    'clip_range': 0.32876417393678736,
    'gae_lambda': 0.8889831140956214
}


def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func


lr_schedule = linear_schedule(1.12e-05)

model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR,
            verbose=1, learning_rate=lr_schedule, **model_params)
model.set_parameters("training/best_model_40000")
model.learn(total_timesteps=TOTAL_TIMESTEP_NUMB,
            callback=callback, reset_num_timesteps=True)


Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to ./logs/PPO_12
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 695      |
|    ep_rew_mean     | -6.36    |
| time/              |          |
|    fps             | 214      |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 5952     |
---------------------------------
超时了
超时了
time steps: 10000 / 2000000
average reward: -3.678999918885529 average time: 508.6 best_reward: 17.499999597668648
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 674         |
|    ep_rew_mean          | -6.15       |
| time/                   |             |
|    fps                  | 109         |
|    iterations           | 2           |
|    time_elapsed         | 108         |
|    total_timesteps      | 11904       |
| train/                  |             |
|    approx_kl            | 0.004

In [ ]:

!ls

Contra-Nes  last1.png  last3.png  monitor_log	  tmp
last0.png   last2.png  logs	  reward_log.csv  training


In [ ]:
env.close() 